# Create  Custom Shape Island Defect on a Slab

Create an island defect on a slab with a custom condition on coordinates extending the existing Class.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Adjust the CoordianteCondition class to define the custom shape of the island defect.
1. Set the parameters in the "Set the parameters" section.
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys
from typing import List


if sys.platform == "emscripten":
    import micropip
    await micropip.install("mat3ra-api-examples @ git+https://github.com/Exabyte-io/esse.git@epic/SOF-7623", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("")

### 1.2. Create custom Coordinate Conditions

In [ ]:
import numpy as np
from mat3ra.made.tools.utils.coordinate import CoordinateCondition

# Custom Coordinate Condition for a torus shape
class CustomCoordinateCondition(CoordinateCondition):
    center: List[float] = [0.5, 0.5, 0.0]
    major_radius: float = 0.5
    minor_radius: float = 0.1 

    def condition(self, coordinate: List[float]) -> bool:
        coord = np.array(coordinate)
        # Calculate the distance from the center in the xy-plane
        xy_distance = np.linalg.norm(coord[:2] - self.center[:2])
        # Calculate the distance from the torus surface
        distance_to_torus = np.sqrt((xy_distance - self.major_radius) ** 2 + (coord[2] - self.center[2]) ** 2)
        return distance_to_torus <= self.minor_radius


### 1.3. Set the parameters


In [ ]:
CENTER_POSITION = [0.5, 0.5, 0.5]  # Center of the torus in crystal coordinates
MAJOR_RADIUS = 0.3  # Major radius of the torus
MINOR_RADIUS = 0.1  # Minor radius of the torus
USE_CARTESIAN_COORDINATES = False # Use Cartesian coordinates for the condition

NUMBER_OF_ADDED_LAYERS = 3  # Number of layers to add to the defect


# Create slab with default parameters if imported material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0,0,1),
    "thickness": 3,
    "vacuum": 5.0,
    "use_orthogonal_c": True,
    "xy_supercell_matrix": [[10, 0], [0, 10]]
}

### 1.4. Initialize the Coordinate Condition

In [ ]:
coordinate_condition = CustomCoordinateCondition(
    center=CENTER_POSITION,
    major_radius=MAJOR_RADIUS,
    minor_radius=MINOR_RADIUS
)

### 1.5. Get input material

In [ ]:
from utils.jupyterlite import get_materials
materials = get_materials(globals())

### 1.6. Preview Slab or create a new one

In [ ]:
from mat3ra.made.tools.build.slab.helpers import create_slab
from utils.visualize import visualize_materials as visualize

slab = materials[0]
if not slab.metadata or not slab.metadata.build:
    print("The material is not a slab. Creating a new slab...")
    slab = create_slab(
        crystal=materials[0],
        miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
        number_of_layers=DEFAULT_SLAB_PARAMETERS["thickness"],
        vacuum= DEFAULT_SLAB_PARAMETERS["vacuum"],
        use_orthogonal_c=DEFAULT_SLAB_PARAMETERS["use_orthogonal_c"],
        xy_supercell_matrix=DEFAULT_SLAB_PARAMETERS["xy_supercell_matrix"]
    )
        
visualize([{"material": slab, "title": "Original material"}])
visualize([{"material": slab, "title": "Original material"}], rotation="-90x")

## 2. Create the Target Material

In [ ]:
from mat3ra.made.tools.build.defect.island.helpers import create_island_defect

slab_with_island = create_island_defect(
    slab=slab,
    condition=coordinate_condition,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES,
    number_of_added_layers=NUMBER_OF_ADDED_LAYERS,
)

## 3. Visualize the Result(s)

In [ ]:
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect"}],
rotation="-90x")

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(slab_with_island)